# High-level Comparison Between the Network Covariance and Characteristic Matrices

In this notebook, we provide a rough comparison between covariance-based and entropy-based topology inference methods.
The goal is to test each method across a range of entangled states to get an idea of their relative performance.


## Key Differences

### Covariance Matrix Advantages

There is a significant performance advantage over the characteristic matrix:
* This can easily be observed by noting the cell execution times, especially when more qubits are considered. 
* The covariance is evaluated from one circuit evaluation $O(1)$, whereas the charcteristic matrix requires a minimum of $O(n^2)$ circuit evaluations where $n$ is the number of qubits. Given clever parallelization in quantum circuits, the characteristic matrix may be evaluated in $O(n)$.
* Optimizing the covariance matrix seems to require significantly fewer steps than needed to successfully infer topology using the characteristic matrix. This means that fewer circuits need to be run.
* The optimization appears to be more robust and less susceptible to local minima. We see this in the fact that the characteristic matrix often fails to infer the GHZ states with more than 3 qubits.


### Characteristic Matrix Advantages

A key difference between the covariance and characteristic matrices is that the covariance matrix seems to always have ones along the diagonal, even if the measured qubit is not correlated with any other qubits. On the contrary, the characteristic matrix measures a zero von neumann entropy if there is no correlation whatsoever.

In [ ]:
import qnetti
import pennylane as qml
from pennylane import numpy as qnp
import qnetvo

print(qnetvo.__version__)
print(qml.__version__)

In [ ]:
def decision_matrix(mat, atol=0.1):
    return qnp.where(qnp.abs(mat) > atol, 1, 0)


def matrix_distance(mat1, mat2):
    return qnp.linalg.norm(qnp.array(mat1) - qnp.abs(qnp.array(mat2)))


def characteristic_matrix_inference(prep_node, expected_mat, **kwargs):
    char_mat = qnetti.qubit_characteristic_matrix(prep_node, **kwargs)
    dec_mat = decision_matrix(char_mat)
    char_dist = matrix_distance(expected_mat, char_mat)

    print("characteristic matrix :\n", char_mat)
    print("decision matrix :\n", dec_mat)
    print("expected matrix :\n", expected_mat)
    print("distance to expected : ", char_dist)


def covariance_matrix_inference(prep_node, num_qubits, expected_mat, meas_wires=None, **kwargs):
    opt_dict = qnetvo.gradient_descent(
        qnetti.qubit_covariance_cost_fn(prep_node, meas_wires=meas_wires),
        qnp.random.rand(3 * num_qubits, requires_grad=True),
        **kwargs,
    )

    cov_mat = qnetti.qubit_covariance_matrix_fn(prep_node, meas_wires=meas_wires)(
        opt_dict["opt_settings"]
    )
    dec_mat = decision_matrix(cov_mat)
    cov_dist = matrix_distance(expected_mat, cov_mat)

    print("covariance matrix :\n", cov_mat)
    print("decision matrix :\n", dec_mat)
    print("expected matrix :\n", expected_mat)
    print("distance to expected : ", cov_dist)

## Two-Qubit GHZ State

In [ ]:
ghz2_mat = qnp.ones((2, 2))


def rot_bell_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])


bell_state_prep_node = qnetvo.PrepareNode(wires=[0, 1], ansatz_fn=rot_bell_state)

In [ ]:
characteristic_matrix_inference(bell_state_prep_node, ghz2_mat, num_steps=10, step_size=0.1)

In [ ]:
covariance_matrix_inference(
    bell_state_prep_node,
    2,
    ghz2_mat,
    step_size=0.2,
    num_steps=6,
    sample_width=1,
    verbose=False,
)

## 3-Qubit GHZ state

In [ ]:
ghz3_mat = qnp.ones((3, 3))


def rot_ghz3_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[2]])


ghz3_state_prep_node = qnetvo.PrepareNode(wires=[0, 1, 2], ansatz_fn=rot_ghz3_state)

In [ ]:
characteristic_matrix_inference(ghz3_state_prep_node, ghz3_mat, num_steps=10, step_size=0.1)

In [ ]:
covariance_matrix_inference(
    ghz3_state_prep_node,
    3,
    ghz3_mat,
    step_size=0.1,
    num_steps=10,
    sample_width=1,
    verbose=False,
)

## 4-Qubit GHZ

In [ ]:
ghz4_mat = qnp.ones((4, 4))


def rot_ghz4_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[2]])
    qml.ArbitraryUnitary([2, 2, 2], wires=[wires[3]])


ghz4_state_prep_node = qnetvo.PrepareNode(wires=[0, 1, 2, 3], ansatz_fn=rot_ghz4_state)

In [ ]:
characteristic_matrix_inference(ghz4_state_prep_node, ghz4_mat, num_steps=30, step_size=0.1)

In [ ]:
covariance_matrix_inference(
    ghz4_state_prep_node,
    4,
    ghz4_mat,
    step_size=0.2,
    num_steps=6,
    sample_width=1,
    verbose=False,
)

# 5-Qubit GHZ

In [ ]:
ghz5_mat = qnp.ones((5, 5))


def rot_ghz5_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[2]])
    qml.ArbitraryUnitary([2, 2, 2], wires=[wires[3]])
    qml.ArbitraryUnitary([1, 2.3, 0.4], wires=[wires[4]])


ghz5_state_prep_node = qnetvo.PrepareNode(wires=[0, 1, 2, 3, 4], ansatz_fn=rot_ghz5_state)

In [ ]:
characteristic_matrix_inference(ghz5_state_prep_node, ghz5_mat, num_steps=30, step_size=0.1)

In [ ]:
covariance_matrix_inference(
    ghz5_state_prep_node, 5, ghz5_mat, step_size=0.1, num_steps=10, sample_width=1, verbose=False
)

# 8-qubit GHZ state

In [ ]:
ghz8_mat = qnp.ones((8, 8))


def rot_ghz8_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[2]])
    qml.ArbitraryUnitary([2, 2, 2], wires=[wires[3]])
    qml.ArbitraryUnitary([1, 2.3, 0.4], wires=[wires[4]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[5]])
    qml.ArbitraryUnitary([2, 2, 2], wires=[wires[6]])
    qml.ArbitraryUnitary([1, 2.3, 0.4], wires=[wires[7]])


ghz8_state_prep_node = qnetvo.PrepareNode(
    num_in=1,
    wires=[0, 1, 2, 3, 4, 5, 6, 7],
    ansatz_fn=rot_ghz8_state,
    num_settings=0,
)

In [ ]:
characteristic_matrix_inference(ghz8_state_prep_node, ghz8_mat, num_steps=30, step_size=0.1)

In [ ]:
covariance_matrix_inference(
    ghz8_state_prep_node,
    8,
    ghz8_mat,
    step_size=0.1,
    num_steps=10,
    sample_width=1,
    verbose=False,
)

# 8 qubit graph state

In [ ]:
# This expected matrix is not known with certainty, but agrees with results
graph8_mat = qnp.array(
    [
        [1, 0, 0, 1, 0, 0, 0, 0],
        [0, 1, 1, 0, 0, 0, 0, 0],
        [0, 1, 1, 0, 0, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1],
    ]
)

graph8_state_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7],
    ansatz_fn=qnetvo.graph_state_fn([[0, 1], [1, 2], [0, 3], [6, 7]]),
)

In [ ]:
characteristic_matrix_inference(graph8_state_prep_node, graph8_mat, num_steps=60, step_size=0.02)

In [ ]:
covariance_matrix_inference(
    graph8_state_prep_node,
    8,
    graph8_mat,
    step_size=0.05,
    num_steps=30,
    sample_width=1,
    verbose=False,
)

## 8-Qubit Graph State with Frustrated CNOT arrangement

In [ ]:
# This expected matrix is not known with certainty, but agrees with results
graph8_cyclic_mat = qnp.array(
    [
        [1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1],
    ]
)

graph8_cyclic_state_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7],
    ansatz_fn=qnetvo.graph_state_fn([[0, 1], [1, 2], [2, 0], [6, 7]]),
)

In [ ]:
characteristic_matrix_inference(
    graph8_cyclic_state_prep_node, graph8_cyclic_mat, num_steps=60, step_size=0.02
)

In [ ]:
covariance_matrix_inference(
    graph8_cyclic_state_prep_node,
    8,
    graph8_cyclic_mat,
    step_size=0.05,
    num_steps=30,
    sample_width=1,
    verbose=False,
)

## 12-Qubit Graph State

In [ ]:
graph12_mat = qnp.array(
    [
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
    ]
)
graph12_state_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    ansatz_fn=qnetvo.graph_state_fn([[0, 1], [1, 2], [6, 7], [4, 5], [4, 9], [4, 11], [11, 10]]),
)

In [ ]:
characteristic_matrix_inference(graph12_state_prep_node, graph12_mat, num_steps=60, step_size=0.02)

In [ ]:
covariance_matrix_inference(
    graph12_state_prep_node,
    12,
    graph12_mat,
    num_steps=20,
    step_size=0.05,
    verbose=False,
    sample_width=1,
)

## A bunch of different entangled states

In [ ]:
multi_ent_mat_full = qnp.zeros((8, 8))
multi_ent_mat = qnp.array(
    [
        [1, 1, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 1],
    ]
)


def fn(settings, wires):
    qnetvo.shared_coin_flip_state([qnp.pi / 3], wires=[wires[0], wires[1], wires[7]])
    qnetvo.ghz_state([], wires=wires[2:4])
    qnetvo.W_state([], wires=wires[4:7])


multi_ent_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7],
    ansatz_fn=fn,
)

In [ ]:
characteristic_matrix_inference(
    multi_ent_prep_node, multi_ent_mat_full, num_steps=50, step_size=0.1
)

In [ ]:
covariance_matrix_inference(
    multi_ent_prep_node,
    7,
    multi_ent_mat,
    meas_wires=[0, 1, 2, 3, 4, 5, 6],
    num_steps=20,
    step_size=0.05,
    verbose=False,
    sample_width=1,
)